# Real Estate Listing Generator

This notebook generates realistic real estate listings for properties in Polish cities using OpenAI's GPT model with structured output. It creates comprehensive property descriptions with pricing, features, and market positioning.

## Setup and Imports

Load required libraries for OpenAI integration, structured data handling, and environment configuration.


In [21]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from pydantic import BaseModel, Field
from typing import List
from enum import Enum
import json
import random

# Load environment variables from .env file
load_dotenv()


True

## Data Model Definition

Define the structure for real estate listings using Pydantic to ensure consistent, validated output from the AI model.


In [22]:
# Define Enums for controlled values
class UrbanLevel(str, Enum):
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class TransportOption(str, Enum):
    PUBLIC_TRANSPORT = "public transport"
    BIKE_LINES = "bike lines"
    HIGHWAY = "highway"
    AIRPORT = "airport"

# Define Pydantic model for structured output
class RealEstateListing(BaseModel):
    title: str = Field(description="A catchy but realistic title for the property listing")
    description: str = Field(description="3-5 sentences describing the property itself")
    neighborhood_description: str = Field(description="2-3 sentences about the surrounding area and amenities")
    
    # Property details
    bedrooms: int = Field(description="Number of bedrooms")
    bathrooms: int = Field(description="Number of bathrooms")
    size_sqm: int = Field(description="Property size in square meters")
    property_type: str = Field(description="Type of property (e.g., apartment, house, loft)")
    
    # Location details
    city: str = Field(description="City where the property is located")
    neighborhood: str = Field(description="Specific neighborhood or district")
    urban_level: UrbanLevel = Field(description="Urban level of the neighborhood")
    transport_options: List[TransportOption] = Field(description="Transport options available in the neighborhood")
    
    # Pricing
    price: int = Field(description="Price of the property")
    
    # Features and amenities
    key_features: List[str] = Field(description="List of key property features and amenities")
    lifestyle_benefits: List[str] = Field(description="List of lifestyle benefits this property offers")
    
    # Market positioning
    target_buyer: str = Field(description="Description of ideal buyer for this property")
    market_positioning: str = Field(description="How this property positions in the local market")


## AI Model Configuration

Initialize the OpenAI GPT model with structured output capability and define the prompt template for generating property listings.


In [23]:
# Initialize OpenAI GPT model with structured output
listing_llm = ChatOpenAI(
    model="gpt-4.1",
    api_key=os.getenv('OPENAI_API_KEY'),
    temperature=1.3 # higher temperature for more creative output
).with_structured_output(RealEstateListing)

prompt_template = """
You are a creative real estate copywriter.

Generate a comprehensive real estate listing for a property in {city}, located in the neighborhood called {neighborhood}. The neighborhood has {urban_level} urban level and {transportation} transportation options.
It should be a {property_type} with {bedrooms} bedrooms and {bathrooms} bathrooms, and a realistic random size reflecting the property type and number of rooms.
The target price range is {price_low}-{price_high} PLN.

Tone: {tone}.
Lifestyle focus (suggested): {lifestyle}.
Main features (suggested): {features}.
Neighborhood vibe (suggested): {vibe_keywords}.

Please provide all the required fields:
- Basic property details (bedrooms, bathrooms, size, type)
- Location information (city, country, neighborhood, urban level, transportation options)
- Pricing details (price range and currency)
- Key features as a list of strings
- Lifestyle benefits as a list of strings
- Target buyer description
- Market positioning description

Constraints:
- All listings are in Poland; currency is PLN; language is English.
- Keep sizes/rooms realistic for the property type.
- **Adjustment rule (short):** Treat suggested features/lifestyle/vibe as guidance—keep what fits the property type and neighborhood, skip or replace items that don't, so all details remain consistent.
- For urban_level, use one of: "high", "medium", "low"
- For transport_options, provide a list of strings from: "public transport", "bike lines", "highway", "airport"
"""

## Property Data Pools

Define comprehensive datasets including Polish cities, property types, pricing ranges, and feature pools to generate diverse and realistic property listings.


In [24]:
cities = {
    "Warsaw": [
        {"name": "Mokotow", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY, TransportOption.AIRPORT]},
        {"name": "Zoliborz", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Wola", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Ursynow", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY, TransportOption.AIRPORT]},
        {"name": "Ochota", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.AIRPORT]}
    ],
    "Krakow": [
        {"name": "Kazimierz", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Podgorze", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Bronowice", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY, TransportOption.AIRPORT]},
        {"name": "Pradnik Bialy", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]},
        {"name": "Debniki", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]}
    ],
    "Gdansk": [
        {"name": "Wrzeszcz", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY, TransportOption.AIRPORT]},
        {"name": "Oliwa", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Srodmiescie", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Zaspa", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Orunia", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]}
    ],
    "Wroclaw": [
        {"name": "Krzyki", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Stare Miasto", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Psie Pole", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]},
        {"name": "Oltaszyn", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.HIGHWAY]},
        {"name": "Nadodrze", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]}
    ],
    "Poznan": [
        {"name": "Jezyce", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Grunwald", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Wilda", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Lazarz", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Stare Miasto", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]}
    ],
    "Lodz": [
        {"name": "Srodmiescie", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Polesie", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Baluty", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]},
        {"name": "Gorna", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]}
    ],
    "Sopot": [
        {"name": "Centrum", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Kamienny Potok", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Karlikowo", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]}
    ],
    "Zakopane": [
        {"name": "Centrum", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]},
        {"name": "Koscielisko", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]},
        {"name": "Pardalowka", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Katowice": [
        {"name": "Ligota", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]},
        {"name": "Dab", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES, TransportOption.HIGHWAY]},
        {"name": "Brynow", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]},
        {"name": "Srodmiescie", "urban_level": UrbanLevel.HIGH, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.BIKE_LINES]}
    ],
    "Kazimierz Dolny": [
        {"name": "Rynek", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Ustka": [
        {"name": "Port District", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY, TransportOption.BIKE_LINES]}
    ],
    "Elk": [
        {"name": "Lake District", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Krosno": [
        {"name": "Old Town", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Swidnica": [
        {"name": "Historic Center", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]}
    ],
    "Suwałki": [
        {"name": "Polnocne", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Zamosc": [
        {"name": "Stare Miasto", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]}
    ],
    "Leszno": [
        {"name": "Zatorze", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]}
    ],
    "Plock": [
        {"name": "Radziwie", "urban_level": UrbanLevel.MEDIUM, "transportation": [TransportOption.PUBLIC_TRANSPORT, TransportOption.HIGHWAY]}
    ],
    "Nowy Sacz": [
        {"name": "Suburb", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Pila": [
        {"name": "Suburb", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ],
    "Chelm": [
        {"name": "Dyrekcja", "urban_level": UrbanLevel.LOW, "transportation": [TransportOption.HIGHWAY]}
    ]
}

property_types = ["apartment", "house", "loft", "duplex", "studio"]

tones = [
    "luxury premium style", "family-friendly and warm",
    "eco-conscious modern", "young professional urban",
    "historic charm", "minimalist Scandinavian", "artistic and creative"
]

lifestyles = [
    "families with children", "remote workers", "retirees",
    "first-time buyers", "young couples", "entrepreneurs",
    "students", "digital nomads", "expats",
    "property investors", "eco-conscious buyers", "luxury seekers",
    "professionals relocating for work",
    "artists and creatives", "pet owners", "long-term renters seeking stability"
]

features_pool = [
    "modern kitchen", "open floor plan", "underfloor heating",
    "air conditioning", "smart home system", "energy-efficient windows",
    "fireplace", "hardwood floors", "built-in wardrobes",
    "walk-in closet", "panoramic windows", "soundproof walls",
    "laundry room", "storage room", "home office space",
    "dining area with natural light", "ensuite bathroom", "guest bathroom",
    "modern lighting system", "security system", "balcony",
    "terrace", "rooftop deck", "private garden",
    "shared courtyard", "fenced yard", "outdoor dining area",
    "barbecue zone", "children's playground area", "landscaped surroundings",
    "automatic gate", "private parking space", "underground garage",
    "bicycle storage", "EV charging station", "solar panels",
    "rainwater collection system", "motion-sensor exterior lights", "heated driveway",
    "patio doors opening to garden", "elevator access", "24/7 security",
    "video intercom", "fitness center", "swimming pool access",
    "shared sauna", "residents' lounge", "co-working space",
    "reception desk", "storage locker", "pets allowed",
    "recycling facilities", "wheelchair accessibility", "modern façade design",
    "newly renovated interior", "well-maintained common areas", "energy-efficient building",
    "quiet neighborhood setting", "close to public transport", "shopping and dining nearby"
]

vibe_pool = [
    "quiet atmosphere", "family-friendly", "green surroundings",
    "close to city center", "well-connected by public transport", "bike-friendly",
    "safe neighborhood", "modern development", "historic charm",
    "vibrant local life", "close to schools", "near parks and playgrounds",
    "shopping and dining nearby", "artistic community", "tech-oriented area",
    "cultural district", "peaceful suburban vibe", "lively urban energy",
    "close to waterfront", "mountain views", "community-focused area",
    "good access to highways", "near business district", "balanced lifestyle",
    "pet-friendly environment", "active outdoor lifestyle", "eco-conscious community",
    "trendy and dynamic", "growing neighborhood", "relaxed pace of life"
]

price_by_type = {
    "studio": (300000, 550000),
    "apartment": (400000, 1500000),
    "loft": (600000, 1800000),
    "house": (700000, 2500000),
    "duplex": (900000, 2000000)
}

bedrooms_by_type = {
    "studio": (1, 1),
    "apartment": (1, 4),
    "loft": (1, 3),
    "house": (3, 6),
    "duplex": (3, 5)
}

bathrooms_by_type = {
    "studio": (1, 1),
    "apartment": (1, 2),
    "loft": (1, 2),
    "house": (2, 3),
    "duplex": (2, 3)
}

def get_randonm_property_params():
    city = random.choice(list(cities.keys()))
    neighborhood_data = random.choice(cities[city])
    neighborhood = neighborhood_data["name"]
    urban_level = neighborhood_data["urban_level"]
    transportation = neighborhood_data["transportation"]
    property_type = random.choice(property_types)
    bedrooms = random.randint(bedrooms_by_type[property_type][0], bedrooms_by_type[property_type][1])
    bathrooms = random.randint(bathrooms_by_type[property_type][0], bathrooms_by_type[property_type][1])
    price_low, price_high = price_by_type[property_type]
    price = random.randint(price_low, price_high)
    tone = random.choice(tones)
    lifestyle = random.choice(lifestyles)
    features = random.sample(features_pool, random.randint(3, 6))
    vibe_keywords = random.sample(vibe_pool, random.randint(2, 4))

    return {
        "city": city,
        "neighborhood": neighborhood,
        "urban_level": urban_level.value,  # Convert enum to string for prompt
        "transportation": [t.value for t in transportation],  # Convert enum list to string list for prompt
        "property_type": property_type,
        "bedrooms": bedrooms,
        "bathrooms": bathrooms,
        "price_low": price_low,
        "price_high": price_high,
        "price": price,
        "tone": tone,
        "lifestyle": lifestyle,
        "features": features,
        "vibe_keywords": vibe_keywords
    }

print(json.dumps(get_randonm_property_params(), indent=2))
    

{
  "city": "Pila",
  "neighborhood": "Suburb",
  "urban_level": "low",
  "transportation": [
    "highway"
  ],
  "property_type": "house",
  "bedrooms": 6,
  "bathrooms": 2,
  "price_low": 700000,
  "price_high": 2500000,
  "price": 2222933,
  "tone": "artistic and creative",
  "lifestyle": "artists and creatives",
  "features": [
    "security system",
    "storage locker",
    "rainwater collection system",
    "soundproof walls",
    "built-in wardrobes",
    "barbecue zone"
  ],
  "vibe_keywords": [
    "eco-conscious community",
    "artistic community",
    "quiet atmosphere",
    "tech-oriented area"
  ]
}


## Generate Property Listing

Create a complete real estate listings by combining random property parameters with the AI model to produce a structured, realistic property description.


In [ ]:
raw_data_path = os.path.join(os.getcwd(), "../data/raw/")

listing_count = 200

for i in range(1, listing_count+1):

    property_params = get_randonm_property_params()
    formatted_prompt = prompt_template.format(**property_params)
    message = HumanMessage(content=formatted_prompt)

    filename = f"listing_{i:03d}.json"
    filepath = os.path.join(raw_data_path, filename)
    print(f"Generating {filename} [{i}/{listing_count}]... ", end="", flush=True)
    if os.path.exists(filepath):
        print(f"skipping because the file already exists")
        continue

    # generate listing
    property_params = get_randonm_property_params()
    response = listing_llm.invoke([message])

    # save listing
    with open(filepath, "w") as f:
        json.dump(response.model_dump(), f, indent=2)
  
    print("ok")
   


Generating listing_001.json [1/200]... skipping because the file already exists
Generating listing_002.json [2/200]... skipping because the file already exists
Generating listing_003.json [3/200]... skipping because the file already exists
Generating listing_004.json [4/200]... skipping because the file already exists
Generating listing_005.json [5/200]... skipping because the file already exists
Generating listing_006.json [6/200]... skipping because the file already exists
Generating listing_007.json [7/200]... skipping because the file already exists
Generating listing_008.json [8/200]... skipping because the file already exists
Generating listing_009.json [9/200]... skipping because the file already exists
Generating listing_010.json [10/200]... skipping because the file already exists
Generating listing_011.json [11/200]... skipping because the file already exists
Generating listing_012.json [12/200]... skipping because the file already exists
Generating listing_013.json [13/200].